# permittivity-calc Tutorial

Everything you need to know about using permittivity-calc: [github.com/boivinalex/permittivity-calc](github.com/boivinalex/permittivity-calc).

Housekeeping
-----------
For information on what permittivity-calc is for and how to install it see the README on [github](github.com/boivinalex/permittivity-calc).

Issues with or questions about this tutorial or permittivity-calc itself can be reported on the [issue tracker](github.com/boivinalex/permittivity-calc/issues).

Introduction
-----------
permittivity-calc works by storing input S-parameter data and associated processed data in a [Python Class](docs.python.org/3/tutorial/classes.html) called AirlineData. The definition of AirlineData is located in sparam_data.py. When data is input into permittivity-calc, a new instance of the AirlineData Class is created.

The AirlineData Class contains several data attributes and methods. 

Data attributes store the raw S-parameter data, processed (complex permittivity) data, as well as variables. Some variables are mandatory such as the length of the transmission line (L), and some are optional such as the bulk density of the sample measured (bulk_density).

Methods are functions which either automatically process the input data, can be called to process data if an argument is given to the AirlineData Class Instance, or must be called manually. Methods are either private methods not meant to be accessed by the user or public methods meant to be used by the user.

In addition to the class methods, permittivity-calc also has helper functions (public and private) which are used to input data (either manually or via file dialog), to plot both the permittivity data and the raw S-parameter data, and to plot multiple permittivity datasets together.

This tutorial is intended to demonstrate every feature of permittivity-calc starting with basic usage and moving on to more advanced, custom, or minor features.

Cheat sheet
-------------------------------------
List of all data attributes, methods, and functions in permittivity-calc

### Data attributes

The mandatory data attributes are:
- **L**: the length of the transmission line [automatically derived or manually provided]
- **airline_name**: name of the airline, determines the length of the line (set for the GR900-LZ15 15 cm long 50 \Omega reference airline; can be changed to match any transmission line) [must be provided]
- **file**: the file path of the input data [must be provided]
- **corr**: boolean flag which determines if s-parameter data should be corrected using connected de-embedding [automatically derived, default: False]
- **nrw**: boolean flag for using the Nicholson-Ross-Weir method to calculate the complex permittivity [must be provided, default: False]
- **freq, s11, s21, s12, s22**: raw s-parameter and frequency data [automatically derived]
- **avg_dielec, avg_lossfac, avg_losstan**: the dielectric constant, loss factor, and loss tangent calculated from the averaged S-parameters ((S11+S22)/2 and (S21+S12)/2)

The optional data attributes are:
- **name**: a name given for plotting [must be provided]
- **bulk_density**: the bulk density of the sample [must be provided]
- **normalize_density**: flag to normalize the density of the permittivity measurement to a constant value (default: 1.60g/cm^3) [must be provided, default: False]
- **norm_eqn**: flag to set which equation is used to normalize the density [must be provided, default: 'LI']
- **temperature**: temperature of the measurement [must be provided]
- **date**: date of the measurement [must be provided]
- **solid_dielec**: dielectric constant of the solid sample being measured [must be provided]
- **solid_losstan**: loss tangent of the solid sample being measured [must be provided]
- **particle_diameter**: average particle diameter of the sample [must be provided]
- **particle_density**: the solid density if the (powdered) sample [must be provided]
- **airline_dimensions**: dimensions of the transmission line. [automatically derived]

Conditional attributes (only generated if the appropriate flags or data attributes are given):
- **forward_dielec, reverse_dielec, forward_lossfac, reverse_lossfac, forward_losstan, reverse_losstan**: forward and reverse dielectric constant, loss factor, and loss tangent, calculated from S11 and S21 (forward) and S22 and S12 [automatically derived if nrw=False]
- **Lcorr**: corrected transmission line length [automatically derived if corr=True]
- **corr_s11, corr_s21, corr_s12, corr_s22**: corrected raw S-parameters [automatically derived if corr=True]
- **corr_avg_dielec, corr_avg_lossfac, corr_avg_losstan**: dielectric constant, loss factor, and loss tangent calculated from the corrected (de-embedded) averaged S-parameters
- **mu**: complex magnetic permittivity [automatically derived if nrw=True]
- **corr_mu**: complex corrected magnetic permittivity [automatically derived if nrw=True and corr=True]
- **bcorr_dielec, bcorr_losstan**: boundary corrected dielectric constant and loss tangent [automatically derived if solid_dielec, particle_diameter, particle_density, and bulk_density are given]
- **norm_dielec, norm_lossfac, norm_losstan**: the bulk density normalized dielectric constant, loss factor, and loss tangent [automatically derived if bulk_density is given and normalize_density=True]

### Methods

The public methods are:
- **draw_plots**: plot the calculated permittivity
- **s_param_plot**: plot the raw S-parameters

The private methods are:
- **\__repr\__**: a [Python Class Special Method](docs.python.org/3/reference/datamodel.html#object.__repr__) which generates a "a valid Python expression that could be used to recreate an object with the same value"
- **\__str\__**: a [Python Class Special Method](docs.python.org/3/reference/datamodel.html#object.__str__) which generates a human readable summary of the data attributes stored in an AirlineData instance
- **_unpack**: unpack input data into freq, s11, s21, s12, s22
- **_dims**: Uses airline_anme to determine the dimensions of the transmission line (set for the GR900-LZ15 15 cm long 50 \Omega reference airline; can be changed to match any transmission line)
- **_permittivity_calc**: calculate the complex permittivity using the New Non-iterative method
- **_calc_uncertainties**: propagate uncertainties in the permittivity (if provided in S-parameters)
- **_de_embed**: perform connector de-embedding and produce corrected S-parameters if corr=True
- **_boundary_correct**: correct the calculated permittivity for boundary effects in the transmission line (requires that solid_dielec, particle_diameter, particle_density, and bulk_density are given)
- **_air_gap_correction**: correct for the air gap in the transmission line if measuring a solid sample

### Functions

The public functions are:
- **get_METAS_data**: function used to input S-parameter data from a .txt file. Stores the data as a numpy array and determines the length of the transmission line based on the name given. Default length values in get_METAS_data are for the the GR900-LZ15 line. Can be given a custom length. The AirlineData Class expect arguments returned by get_METAS_data
- **perm_compare**: takes a list of AirlineData Class Instances and plots them together
- **multiple_meas**: used to open multiple files at once. Given any .txt file in a folder, will try to open all .txt files in that folder and process them as S-parameter data. Each .txt file generates its own AirlineData Class Instance and all the data are plotted together using perm_compare
- **run_default**: the simplest way to use permittivity-calc. Calls get_METAS_data and feeds the output to the AirlineData Class
- **run_example**: produces the example AirlineData instances

The private functions are:
- **_prompt**: produces file dialog window using tkinter. Used to select data file
- **_get_file**: gets the file path and airline name

Tutorial
-------
For simplicity, permittivity-calc is meant to be run from a Python script (run_permittivity_calc.py) which imports all necessary Python packages and scripts in permittivity-calc.

By default, run_permittivity_calc.py also runs on two example files.

In [2]:
run run_permittivity_calc.py

/Users/alex/Google Drive/research_data/Python Scripts/permittivity-calc/data/2.5hrs.txt
Shapiro-Wilk test (W, p-val)
(0.9947804808616638, 0.038624539971351624)
(0.635087251663208, 8.541890412368429e-34)
(0.9311879873275757, 5.407571096532557e-16)
(0.9565504193305969, 2.547074964137641e-12)
Bartlett test (stat, p-val)
BartlettResult(statistic=1.0007977755645656, pvalue=0.31711754650148116)
BartlettResult(statistic=0.046278913186469137, pvalue=0.8296695956146326)
BartlettResult(statistic=0.00074221863813440849, pvalue=0.97826536914214923)
BartlettResult(statistic=2.8312623075874127e-08, pvalue=0.99986574511000492)
Reflection (Mag,Phase) = 0.574297242195, 0.434547156797
Transmission (Mag,Phase) = 0.983765287114, 0.9895463986
/Users/alex/Google Drive/research_data/Python Scripts/permittivity-calc/data/atm.txt
Shapiro-Wilk test (W, p-val)
(0.9175930023193359, 1.4051652900766978e-17)
(0.7564078569412231, 7.762513002073117e-29)
(0.9658198952674866, 1.2962671214200583e-10)
(0.9555260539054871,